<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/2%EC%9E%A5_%EC%96%B4%EC%8A%A4%EC%97%94%EC%A7%84_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2장. 어스엔진 데이터셋 실습



In [6]:
import ee
import geemap

import pandas as pd

In [2]:
# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

## 2.1. 피처 컬렉션

### 2.1.1. 피처 컬렉션 사용 (보호지역)

In [3]:
# 피처 컬렉션 객체 (보호지역)
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")

In [8]:
# 피처 선택 (보호지역 ID == 32712)
wdpa_id = 30712
protected_area = wdpa.filter(ee.Filter.eq('WDPAID', wdpa_id))

# 피처 이름 (변산반도)
area_name = protected_area.first().get('NAME').getInfo()
print("Name:", area_name)

Name: Byeonsanbando


In [16]:
# 피처 컬렉션 출력 (변산반도)
m = geemap.Map(layout={'height':'400px', 'width':'800px'},
                 basemap='Esri.WorldImagery')

m.addLayer(protected_area, {'color': 'green'}, area_name)
m.centerObject(protected_area, 11)
m

Map(center=[35.64739890217585, 126.5763703325834], controls=(WidgetControl(options=['position', 'transparent_b…

### 2.1.2. 피처 컬렉션 사용 (행정구역)

In [18]:
# 피처 컬렉션 객체 (행정구역)
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")

# 피처 선택 ('ADM0_NAME' == '대한민국)
kr = countries.filter(ee.Filter.eq('ADM0_NAME', 'Republic of Korea'))

# 피처 컬렉션 출력 (대한민국)
m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(kr, {}, "Republic of Korea")
m.centerObject(kr, 6)
m

Map(center=[36.34449219000809, 127.80177597242088], controls=(WidgetControl(options=['position', 'transparent_…

## 2.2. 이미지 컬렉션

### 2.2.1. 이미지 사용 (DEM)

In [22]:
# 이미지 객체 (DEM)
srtm = ee.Image("USGS/SRTMGL1_003")
elevation = srtm.select("elevation")
vis_params = {
    "min": 0,
    "max": 5000,
    "palette": [
        "000000",
        "006600",
        "009900",
        "33CC00",
        "996600",
        "CC9900",
        "CC9966",
        "FFFFFF"
    ]
}

# 이미지 출력 (DEM)
m = geemap.Map(layout={"height": "400px", "width": "800px"})
m.addLayer(elevation, vis_params, "Elevation")
m.centerObject(kr, 4)
m

Map(center=[36.34449219000809, 127.80177597242088], controls=(WidgetControl(options=['position', 'transparent_…

### 2.2.2. 이미지 컬렉션 (수목 피복률)

In [24]:
# 이미지 컬렉션 객체 (MOD44B)
modis = ee.ImageCollection('MODIS/006/MOD44B')

# 이미지 선택 (수목 피복률)
filtered_modis = modis.filterDate('2003-01-01', '2020-12-31')
selected_modis = filtered_modis.select('Percent_Tree_Cover')
median_ptc = selected_modis.median()
vis_params = {
    'min': 0,
    'max': 100,
    'palette': ['bbe029', '0a9501', '074b03']}

# 이미지 출력 (수목 피복률)
m = geemap.Map(layout={"height": "400px", "width": "800px"})
m.addLayer(median_ptc, vis_params, "Percent Tree Cover")
m.centerObject(kr, 4)
m

Map(center=[36.34449219000809, 127.80177597242088], controls=(WidgetControl(options=['position', 'transparent_…

### 2.2.3. 이미지 컬렉션 (토지피복지도)

In [48]:
# 이미지 컬렉션 객체 (Sentinel-2 이미지)
image = geemap.dynamic_world_s2(protected_area.geometry(),
                                '2022-01-01', '2023-01-01')
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

# 이미지 출력 (Sentinel-2 이미지)
m = geemap.Map(layout={"height": "400px", "width": "800px"})
m.addLayer(image, vis_params, "Sentinel-2 image")
m.centerObject(protected_area, 11)
m

Map(center=[35.64739890217585, 126.5763703325834], controls=(WidgetControl(options=['position', 'transparent_b…

In [47]:
landcover = geemap.dynamic_world(protected_area.geometry(),
                                 '2022-01-01', '2023-01-01')

# 이미지 출력 (토지피복지도)
m = geemap.Map(layout={"height": "400px", "width": "800px"})
m.addLayer(landcover, {}, 'Land Cover')
m.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
m.centerObject(protected_area, 11)
m

Map(center=[35.64739890217585, 126.5763703325834], controls=(WidgetControl(options=['position', 'transparent_b…